In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'sk-....'
api_key='........'
import os
os.environ['GROQ_API_KEY'] = api_key

In [ ]:
# Optional: add tracing to visualize the agent trajectories
import os
from getpass import getpass

from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq


In [ ]:
import operator
from typing import Any, List, Dict
from typing_extensions import Annotated, TypedDict
from langgraph.graph import StateGraph, END
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
import json 
class State(TypedDict):
    input: str
    responses: str
    feedback: str
    aggregated_response: Annotated[list, operator.add]
    final_answer: str
    iteration: int
    max_iterations: int
    reflection: str

# Initialize models and search tool
proposer_models = [
    ChatGroq(temperature=0, model_name="llama3-70b-8192",),
    ChatGroq(temperature=0, model_name="llama3-8b-8192",),
    ChatGroq(temperature=0, model_name="mixtral-8x7b-32768"),
]

aggregator_model = ChatOpenAI(model="gpt-4o", temperature=0)
reflector_model = ChatOpenAI(model="gpt-4o", temperature=0)



In [ ]:
class Proposer:
    def __init__(self, model: ChatOpenAI, id: str):
        self.model = model
        self.id = id

    def __call__(self, state: State) -> Dict[str, Dict[str, str]]:
        
        prompt = ChatPromptTemplate.from_messages([
            ("human", "Please provide a response to the following prompt: {input}\n\n"
             "The previous response was {response}\n\n"
             "The provided feedbackw was {feedback} \n\n"
                      "Always answer with as much detail as possible.")
        ])
        response = self.model(prompt.format_messages(
            input=state["input"],
            response=state['responses'],
            feedback=state['feedback']
            
        ))
        
        
        
        return {"aggregated_response": [json.dumps({self.id: response.content})]}

In [ ]:
def aggregator(state: State) -> State:
    prompt = ChatPromptTemplate.from_messages([
        #("human", "Given the following responses to the prompt '{input}', please synthesize them into a single, high-quality response:\n\n{responses}")
        ("human", '''
        You have been provided with a set of responses from various open-source models to the  user query '{input}'.
        Your task is to synthesize these responses into a single, high-quality response.
        It is crucial to critically evaluate the information provided in these responses, 
        recognizing that some of it may be biased or incorrect. 
        Your response should not simply replicate the given answers but should offer a refined, accurate, and comprehensive reply to the instruction. Ensure your response is well-structured, coherent, and adheres to the highest standards of accuracy and reliability.
 
        Responses from models:
        \n\n{responses}

''')
    ])
    response = aggregator_model(prompt.format_messages(
        input=state["input"],
        responses="\n\n".join(state["aggregated_response"])
    ))
    state["responses"] = response.content
    return state

In [ ]:
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser


class GradeGeneration(BaseModel):
    """Binary score for relevance check on retrieved documents."""
    score: str = Field(
        description="Is this the correct answer to the question, 'yes' or 'no'"
    )
    feedback: str = Field(
        description="Provided specific feedback for improvement"
    )
        
def reflector(state: State) -> State:
    parser = PydanticOutputParser(pydantic_object=GradeGeneration)

    prompt  = ChatPromptTemplate.from_messages([
            ("human", "Given the following answer to the question: '{input}'\n\n"
            "Answer: {aggregated_response}\n\n"
                      "If the answer is satisfactory and complete, grade it as yes. \n"
    "Provide json object with  feedback string and binary score 'yes' or 'no' score to indicate whether the answer is correct"
            )
        ])
    chain = prompt | reflector_model | parser

    response = chain.invoke({
        'input':state["input"],
        'aggregated_response':state["responses"]
    })
    state["reflection"] = response.score
    state['feedback'] = response.feedback
    state["iteration"] += 1
    return state

In [ ]:

def should_continue(state: State) -> str:
    if state["iteration"] >= state["max_iterations"]:# or "YES" in state["reflection"].upper():
        state["final_answer"] = state["responses"]
        return "end"
    return "refine"

# Create the graph
workflow = StateGraph(State)

# Add nodes
for i, model in enumerate(proposer_models):
    workflow.add_node(f"proposer_{i}", Proposer(model, f"proposer_{i}"))
workflow.add_node("aggregator", aggregator)
workflow.add_node("reflector", reflector)

# Define edges
workflow.set_entry_point("proposer_0")
for i in range(1, len(proposer_models)):
    workflow.add_edge(f"proposer_0", f"proposer_{i}")
    workflow.add_edge(f"proposer_{i}", "aggregator")
workflow.add_edge("aggregator", "reflector")
workflow.add_conditional_edges(
    "reflector",
    should_continue,
    {
        "refine": "proposer_0",
        "end": END
    }
)

In [ ]:
app = workflow.compile()

In [ ]:
def query_moa(question):
    # Run the graph
    initial_state = {
        "input": question,
        "responses": '',
        'feedback':"",
        "aggregated_response": [],
        "final_answer": "",
        "iteration": 0,
        "max_iterations": 3,
        "reflection": ""
    }
    for output in app.stream(initial_state):
        for key, value in output.items():
            print(f"Finished running: {key}:")
    print(value['responses'])
    return value

In [ ]:
value = query_moa("Explain the latest advancements in quantum computing and their potential applications.")

In [ ]:
value=query_moa("Write the snake game in python without any errors")

In [ ]:
value = query_moa("if we lay 5 shirts in the sun and it takes 4 hours to dry, how long would 20 shirts take to dry?")

In [ ]:
value = query_moa('''Maria is staying at a hotel that charges $99.95 per night plus tax for a room. A tax of 8% is applied to the room rate, 
and an additional onetime untaxed fee of $5.00 is charged by the hotel. 
Which of the following represents Maria’s total charge, in dollars, for staying x nights?  (99.95 + 0.08x) + 5  1.08(99.95x) + 5  1.08(99.95x + 5)  1.08(99.95 + 5)x''')

In [ ]:
value =  query_moa('''There are three killers in a room.
Someone enters the room and kills one of them. Nobody leaves the room. How many killers are left in the room? Explain your reasoning step by step.''')

In [ ]:
value =  query_moa('''A marble is put in a glass.
The glass is then turned upside down and put on a table.
Then the glass is picked up and put in a microwave. 
Where's the marble? Explain your reasoning step by step.''')

In [ ]:
value = query_moa('Give me 10 sentences that end in the word Apple.')

In [ ]:
value = query_moa('''It takes one person 5 hours to dig a 10 foot hole in the ground. 
How long would it take 50 people to dig a single 10 foot hole?'

''')